In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dask_ml.preprocessing import Categorizer
from glum import GeneralizedLinearRegressor, TweedieDistribution
from lightgbm import LGBMRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, SplineTransformer, StandardScaler
#import everything here in the masterfile
#kernels sometimes need to be restarted when new lib installed
import dalex as dx

import sys
from pathlib import Path

# Getting project root
project_root = Path.cwd().resolve().parent

# Add the project root directory to sys.path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


from ps3.data import create_sample_split, load_transform
from evaluation.evaluate_predictions import evaluate_predictions
#from ps3.data import train_tweedie, model_pipeline, spline_model
#only if code reworked modularly, currently not needed

In [ ]:
# load data
df = load_transform()
df.head()

In [ ]:
# Checking to ensure IDpol column is unique - we shall carry out split on this
len(df["IDpol"].unique()) == len(df)

In [ ]:
# Train benchmark tweedie model. This is entirely based on the glum tutorial.
#weight = df["Exposure"].values
df["PurePremium"] = df["ClaimAmountCut"] / df["Exposure"]

outcome = "PurePremium"
# TODO: Why do you think, we divide by exposure here to arrive at our outcome variable?
'''
To get a yearly figure so we can compare claims that have a different time under risk
'''


# TODO: use your create_sample_split function here
df_split = create_sample_split(df,"IDpol",0.8)
df_train = df_split[0].copy()
df_test = df_split[1].copy()


categoricals = ["VehBrand", "VehGas", "Region", "Area", "DrivAge", "VehAge", "VehPower"]

predictors = categoricals + ["BonusMalus", "Density"]
glm_categorizer = Categorizer(columns=categoricals)


X_train_t = glm_categorizer.fit_transform(df_train[predictors])
X_test_t = glm_categorizer.transform(df_test[predictors])
y_train_t, y_test_t = df_train[outcome], df_test[outcome]
w_train_t, w_test_t = df_train["Exposure"], df_test["Exposure"]

TweedieDist = TweedieDistribution(1.5)
t_glm1 = GeneralizedLinearRegressor(family=TweedieDist, l1_ratio=1, fit_intercept=True)
t_glm1.fit(X_train_t, y_train_t, sample_weight=w_train_t)


pd.DataFrame(
    {"coefficient": np.concatenate(([t_glm1.intercept_], t_glm1.coef_))},
    index=["intercept"] + t_glm1.feature_names_,
).T

df_test.loc[:,"pp_t_glm1"] = t_glm1.predict(X_test_t)
df_train.loc[:,"pp_t_glm1"] = t_glm1.predict(X_train_t)

print(
    "training loss t_glm1:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_glm1"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_glm1:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_glm1"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "Total claim amount on train set, observed = {}, predicted = {}".format(
        df_train["ClaimAmountCut"].values.sum(),
        np.sum(df_test["Exposure"].values * t_glm1.predict(X_test_t)),
    )
)

In [ ]:
# TODO: Let's add splines for BonusMalus and Density and use a Pipeline.
# Steps: 
# 1. Define a Pipeline which chains a StandardScaler and SplineTransformer. 
#    Choose knots="quantile" for the SplineTransformer and make sure, we 
#    are only including one intercept in the final GLM. 
# 2. Put the transforms together into a ColumnTransformer. Here we use OneHotEncoder for the categoricals.
# 3. Chain the transforms together with the GLM in a Pipeline.
categoricals = ["VehBrand", "VehGas", "Region", "Area", "DrivAge", "VehAge", "VehPower"]
# Let's put together a pipeline
numeric_cols = ["BonusMalus", "Density"]
preprocessor = ColumnTransformer(
    transformers=[
        # TODO: Add numeric transforms here
        ("cat", OneHotEncoder(sparse_output=False, drop="first"), categoricals),
        ("spline", SplineTransformer(knots="quantile"), numeric_cols),
        ("scale", StandardScaler(), numeric_cols)
    ]
)
preprocessor.set_output(transform="pandas")
model_pipeline = Pipeline(
    # TODO: Define pipeline steps here
    [
        ("preprocessing", preprocessor),
        (
            "estimate",
            GeneralizedLinearRegressor(
                family=TweedieDist, l1_ratio=1, fit_intercept=True
            ),
        ),
    ]
)

#incorporate this part later
# let's have a look at the pipeline
model_pipeline

# let's check that the transforms worked
model_pipeline[:-1].fit_transform(df_train)

model_pipeline.fit(df_train, y_train_t, estimate__sample_weight=w_train_t)

pd.DataFrame(
    {
        "coefficient": np.concatenate(
            ([model_pipeline[-1].intercept_], model_pipeline[-1].coef_)
        )
    },
    index=["intercept"] + model_pipeline[-1].feature_names_,
).T

df_test.loc[:,"pp_t_glm2"] = model_pipeline.predict(df_test)
df_train.loc[:,"pp_t_glm2"] = model_pipeline.predict(df_train)

print(
    "training loss t_glm2:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_glm2"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_glm2:  {}".format(
        TweedieDist.deviance(y_test_t, df_test["pp_t_glm2"], sample_weight=w_test_t)
        / np.sum(w_test_t)
    )
)
# Create Boolean mask for test rows 
test = df["sample"] == "test"
print(
    "Total claim amount on test set, observed = {}, predicted = {}".format(
        df["ClaimAmountCut"].values[test].sum(),
        np.sum(df["Exposure"].values[test] * df_test["pp_t_glm2"]),
    )
)


In [ ]:
model_pipeline

In [ ]:
# TODO: Let's use a GBM instead as an estimator.
# Steps
# 1: Define the modelling pipeline. Tip: This can simply be a LGBMRegressor based on X_train_t from before.
# 2. Make sure we are choosing the correct objective for our estimator.
model_pipeline = Pipeline([("estimate", LGBMRegressor(objective="tweedie"))])
model_pipeline.fit(X_train_t, y_train_t, estimate__sample_weight=w_train_t)
df_test.loc[:,"pp_t_lgbm"] = model_pipeline.predict(X_test_t)
df_train.loc[:,"pp_t_lgbm"] = model_pipeline.predict(X_train_t)
print(
    "training loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_lgbm"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_test_t, df_test["pp_t_lgbm"], sample_weight=w_test_t)
        / np.sum(w_test_t)
    )
)
#substantial difference between losses, might overfit for training

In [ ]:
# TODO: Let's tune the LGBM to reduce overfitting.
# Steps:
# 1. Define a `GridSearchCV` object with our lgbm pipeline/estimator. Tip: Parameters for a specific step of the pipeline
# can be passed by <step_name>__param. 

# Note: Typically we tune many more parameters and larger grids,
# but to save compute time here, we focus on getting the learning rate
# and the number of estimators somewhat aligned -> tune learning_rate and n_estimators
cv_u_lgbm = GridSearchCV(
    model_pipeline,
    {
        "estimate__learning_rate": [0.01, 0.02, 0.03, 0.04, 0.05, 0.1],
        "estimate__n_estimators": [50, 100, 150, 200],
    },
    verbose=2,
)
cv_u_lgbm.fit(X_train_t, y_train_t, estimate__sample_weight=w_train_t)

df_test.loc[:,"pp_t_lgbm"] = cv_u_lgbm.best_estimator_.predict(X_test_t)
df_train.loc[:,"pp_t_lgbm"] = cv_u_lgbm.best_estimator_.predict(X_train_t)

print(
    "training loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_lgbm"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_test_t, df_test["pp_t_lgbm"], sample_weight=w_test_t)
        / np.sum(w_test_t)
    )
)

print(
    "Total claim amount on test set, observed = {}, predicted = {}".format(
        df["ClaimAmountCut"].values[test].sum(),
        np.sum(df["Exposure"].values[test] * df_test["pp_t_lgbm"]),
    )
)
#[CV] END estimate__learning_rate=0.1, estimate__n_estimators=200; total time=   2.1s

In [ ]:
# Saving best unconstrained lgbm model
best_unconstrained_lgbm = cv_u_lgbm.best_estimator_

In [ ]:
# Let's compare the sorting of the pure premium predictions
# Source: https://scikit-learn.org/stable/auto_examples/linear_model/plot_tweedie_regression_insurance_claims.html
def lorenz_curve(y_true, y_pred, exposure):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    exposure = np.asarray(exposure)

    # order samples by increasing predicted risk:
    ranking = np.argsort(y_pred)
    ranked_exposure = exposure[ranking]
    ranked_pure_premium = y_true[ranking]
    cumulated_claim_amount = np.cumsum(ranked_pure_premium * ranked_exposure)
    cumulated_claim_amount /= cumulated_claim_amount[-1]
    cumulated_samples = np.linspace(0, 1, len(cumulated_claim_amount))
    return cumulated_samples, cumulated_claim_amount


fig, ax = plt.subplots(figsize=(8, 8))

for label, y_pred in [
    ("LGBM", df_test["pp_t_lgbm"]),
    ("GLM Benchmark", df_test["pp_t_glm1"]),
    ("GLM Splines", df_test["pp_t_glm2"]),
]:
    ordered_samples, cum_claims = lorenz_curve(
        df_test["PurePremium"], y_pred, df_test["Exposure"]
    )
    gini = 1 - 2 * auc(ordered_samples, cum_claims)
    label += f" (Gini index: {gini: .3f})"
    ax.plot(ordered_samples, cum_claims, linestyle="-", label=label)

# Oracle model: y_pred == y_test
ordered_samples, cum_claims = lorenz_curve(
    df_test["PurePremium"], df_test["PurePremium"], df_test["Exposure"]
)
gini = 1 - 2 * auc(ordered_samples, cum_claims)
label = f"Oracle (Gini index: {gini: .3f})"
ax.plot(ordered_samples, cum_claims, linestyle="-.", color="gray", label=label)

# Random baseline
ax.plot([0, 1], [0, 1], linestyle="--", color="black", label="Random baseline")
ax.set(
    title="Lorenz Curves",
    xlabel="Fraction of policyholders\n(ordered by model from safest to riskiest)",
    ylabel="Fraction of total claim amount",
)
ax.legend(loc="upper left")
plt.plot()

In [ ]:
#START OF PROBLEM SET 4
#EX1 MONOTONOCITY CONSTRAINTS
#EX1.1 Plotting average claims
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
bin_edges = list(range(50, 260, 10))
df['BonusMalusBin'] = pd.cut(df['BonusMalus'], bins=bin_edges, include_lowest=True) #need to specify true, otherwise 50 exluded as first interval open
aggregated = df.groupby("BonusMalusBin").apply(
    lambda x: np.average(x["PurePremium"])
).reset_index(name='WeightedAvgClaim')

aggregated['BinMidpoint'] = aggregated['BonusMalusBin'].apply(lambda b: (b.left + b.right) / 2)

plt.figure(figsize=(10, 6))
plt.plot(aggregated['BinMidpoint'], aggregated['WeightedAvgClaim'], marker='o', linestyle='-', linewidth=2)
#custom xaxis for interpretability
plt.xlim(40, 230)  
plt.xticks(range(50, 230, 10))
plt.xticks(fontsize=10)

#generating titles
plt.title('Weighted Average Claim Amount by BonusMalus', fontsize=14, weight='bold')
plt.xlabel('BonusMalus (Binned)', fontsize=12)
plt.ylabel('Weighted Average Claim', fontsize=12)
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.yticks(fontsize=10)

# Show the plot
plt.tight_layout()
plt.show()

#What will/could happen if we do not include a monotonicity constraint?
#here breaks are observed and claim amount is generally linearly increasing, but there are notable breaks
#130-140 is the first peak and as we move to groups with less and less representation the average claim becomes more volatile
#last 7 bins dont include much more than 50 people total and last 3 bins include 4 people overall
#except for peak at 130/140 results for well represented values as expected

In [ ]:
#providing numbers of people in each bin
binned_data = pd.cut(df["BonusMalus"], bins=bin_edges, include_lowest=True)
bin_counts = binned_data.value_counts().sort_index()
print(bin_counts)

In [ ]:
#creating placeholder array for monotonic constraints 
num_features = X_train_t.shape[1]
monotonic_constraints = [0] * num_features
monotonic_constraints[7] = 1 #(9 predictors in total and listed from 0 so bonus malus index 7)

numeric_cols = ["BonusMalus", "Density"]

constrained_lgbm = Pipeline(
    [
        ("preprocessing", preprocessor),
        (
            "estimate",
            GeneralizedLinearRegressor(
                family=TweedieDist, l1_ratio=1, fit_intercept=True
            ),
        ),
    ]
)
#renamed pipeline
constrained_lgbm = Pipeline([("estimate", LGBMRegressor(objective="tweedie", monotone_constraints=monotonic_constraints))])
constrained_lgbm.fit(X_train_t, y_train_t, estimate__sample_weight=w_train_t)
df_test.loc[:,"pp_t_lgbm"] = constrained_lgbm.predict(X_test_t)
df_train.loc[:,"pp_t_lgbm"] = constrained_lgbm.predict(X_train_t)
print(
    "training loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_lgbm"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_test_t, df_test["pp_t_lgbm"], sample_weight=w_test_t)
        / np.sum(w_test_t)
    )
)
#different results than the regular lgbm, which is encouraging
#greater loss for both expected, as we have introduced further constraints on our model

In [ ]:
#tuning our constrained lgbm given different learning rates, different number of estimators and 
cv_c_lgbm = GridSearchCV(
    constrained_lgbm,
    {
        "estimate__learning_rate": [0.01, 0.02, 0.03, 0.04, 0.05, 0.1],
        "estimate__n_estimators": [50, 100, 150, 200],
    }
)
cv_c_lgbm.fit(X_train_t, y_train_t, estimate__sample_weight=w_train_t)

df_test.loc[:,"pp_t_lgbm_constrained"] = cv_c_lgbm.best_estimator_.predict(X_test_t)
df_train.loc[:,"pp_t_lgbm_constrained"] = cv_c_lgbm.best_estimator_.predict(X_train_t)

print(
    "training loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_train_t, df_train["pp_t_lgbm_constrained"], sample_weight=w_train_t)
        / np.sum(w_train_t)
    )
)

print(
    "testing loss t_lgbm:  {}".format(
        TweedieDist.deviance(y_test_t, df_test["pp_t_lgbm_constrained"], sample_weight=w_test_t)
        / np.sum(w_test_t)
    )
)

print(
    "Total claim amount on test set, observed = {}, predicted = {}".format(
        df["ClaimAmountCut"].values[test].sum(),
        np.sum(df["Exposure"].values[test] * df_test["pp_t_lgbm_constrained"]),
    )
)
#higher training loss but a lower testing loss, seems like an improvement

In [ ]:
# Saving best constrained lgbm model
best_constrained_lgbm = cv_c_lgbm.best_estimator_

In [ ]:
#EX2 LEARNING CURVE
#EX2.1 Re-fit the best constrained lgbm estimator
import lightgbm as lgb

best_constrained_lgbm = cv_c_lgbm.best_estimator_
lgbm_model = best_constrained_lgbm.named_steps['estimate']
eval_set = [(X_train_t, y_train_t), (X_test_t, y_test_t)]

lgbm_model.fit(
    X_train_t, y_train_t, 
    sample_weight=w_train_t, 
    eval_set=eval_set,
    eval_metric='l2',
    eval_names=['train', 'test']
)

In [ ]:
# Plot
lgb.plot_metric(lgbm_model, metric='l2')
plt.title('Learning Curve - Constrained LGBM Regressor')
plt.show()

The training and tesing loss values are high and the learning curves are almost horizontal, this may suggest underfitting. The constrained model hasn't learned the relationship in the data well.

In [ ]:
#EX3: Metrics function

def compare_models(X_train_t, y_train_t, X_test_t, y_test_t, sample_weight, monotonic_constraints=None):
    # Models
    constrained_lgbm = Pipeline([
        ("estimate", lgb.LGBMRegressor(objective="tweedie", monotone_constraints=monotonic_constraints))
    ])
    unconstrained_lgbm = Pipeline([("estimate", lgb.LGBMRegressor(objective="tweedie"))])

    # Train models
    constrained_lgbm.fit(X_train_t, y_train_t, estimate__sample_weight=sample_weight)
    unconstrained_lgbm.fit(X_train_t, y_train_t, estimate__sample_weight=sample_weight)

    # Predictions
    y_pred_constrained = constrained_lgbm.predict(X_test_t)
    y_pred_unconstrained = unconstrained_lgbm.predict(X_test_t)

    # Evaluate predictions
    evaluation_constrained = evaluate_predictions(y_test_t, y_pred_constrained, sample_weight=sample_weight)
    evaluation_unconstrained = evaluate_predictions(y_test_t, y_pred_unconstrained, sample_weight=sample_weight)

    return {
        "Constrained Model Evaluation": evaluation_constrained,
        "Unconstrained Model Evaluation": evaluation_unconstrained
    }

# Results
results = compare_models(X_train_t, y_train_t, X_test_t, y_test_t, w_train_t, monotonic_constraints=monotonic_constraints)

print("Constrained Model Evaluation:")
print(results["Constrained Model Evaluation"])

print("\nUnconstrained Model Evaluation:")
print(results["Unconstrained Model Evaluation"])

In [ ]:
#EX4
# Creating explainer for unconstrained LGBM
best_unconstrained_lgbm_exp = dx.Explainer(best_unconstrained_lgbm, X_train_t, y_train_t, label="Unconstrained LGBM")

# Creating explainer for constrained LGBM
best_constrained_lgbm_exp = dx.Explainer(best_constrained_lgbm, X_train_t, y_train_t, label="Constrained LGBM")

In [ ]:
X_train_t.info()

In [ ]:
# unconstrained LGBM
pd_lgbm_unconstrained = best_unconstrained_lgbm_exp.model_profile()
# constrained LGBM
pd_lgbm_constrained = best_constrained_lgbm_exp.model_profile()

In [ ]:
# Plots for unconstrained LGBM
pd_lgbm_unconstrained.plot()

In [ ]:
# Plots for constrained LGBM
pd_lgbm_constrained.plot()
#this makes a lot of sense due to monotonicity constraints

In [ ]:
# unconstrained LGBM
pd_lgbm_unconstrained_cat = best_unconstrained_lgbm_exp.model_profile(variable_type="categorical")
# constrained LGBM
pd_lgbm_constrained_cat = best_constrained_lgbm_exp.model_profile(variable_type="categorical")

In [ ]:
# Plots for unconstrained LGBM
pd_lgbm_unconstrained_cat.plot()

In [ ]:
# Plots for constrained LGBM

pd_lgbm_constrained_cat.plot()

In [ ]:
# EX 5.1
best_constrained_lgbm_exp
best_unconstrained_lgbm_exp

In [ ]:
# EX 5.2
X_test_t_row1 = X_test_t[0:1]

pp_best_constrained_lgbm_exp = best_constrained_lgbm_exp.predict_parts(
    new_observation=X_test_t_row1,
    type="shap",
    label="Contribution of each feature to the constrained prediction of the 1st observation"
    )

pp_best_unconstrained_lgbm_exp = best_unconstrained_lgbm_exp.predict_parts(
    new_observation=X_test_t_row1,
    type="shap",
    label="Contribution of each feature to the unconstrained prediction of the 1st observation"
    )


In [ ]:
pp_best_constrained_lgbm_exp.plot()

In [ ]:
pp_best_unconstrained_lgbm_exp.plot()

The contribution of BonusMalus to the predicted value differs across our constrained and unconstrained model which we might expect given the monotonicity constraint imposed in the constrained model which makes the contribution at this relatively low BonusMalus level negative. The direction of the contribution of region and area also varies across the two models perhaps suggesting that their overall relationship to the dependent variable is weak and thus their values adjust readily to compensate for the change in the contribution of BonusMalus.